In [3]:
import json

In [15]:
# load the data json with info about audio occurences
data_json = '../frames/complete.json'
data_info = json.load(open(data_json)) 
print(data_info.keys())

games = ['mspacman', 'spaceinvaders', 'revenge', 'seaquest', 'enduro']

frames = data_info['mspacman_JE5W3X5P3T_1'].keys()
print(len(frames))

audio_det = []
reward = []
confidence = []
frame_num = []
for i in frames:
    audio_det.append(data_info['mspacman_JE5W3X5P3T_1'][i]['audio'])
    reward.append(data_info['mspacman_JE5W3X5P3T_1'][i]['reward'])
    confidence.append(data_info['mspacman_JE5W3X5P3T_1'][i]['confidence'])
    frame_num.append(data_info['mspacman_JE5W3X5P3T_1'][i]['frame_num'])
    
# print(reward)

dict_keys(['mspacman_JE5W3X5P3T_1', 'mspacman_JE5W3X5P3T_2', 'mspacman_JE5W3X5P3T_3', 'mspacman_JE5W3X5P3T_4', 'mspacman_JE5W3X5P3T_5', 'mspacman_JE5W3X5P3T_6', 'spaceinvaders_X549THSLUZ_1', 'spaceinvaders_X549THSLUZ_2', 'spaceinvaders_X549THSLUZ_3', 'spaceinvaders_X549THSLUZ_4', 'spaceinvaders_X549THSLUZ_5', 'spaceinvaders_X549THSLUZ_6', 'spaceinvaders_X549THSLUZ_7', 'spaceinvaders_X549THSLUZ_8', 'spaceinvaders_X549THSLUZ_9', 'spaceinvaders_X549THSLUZ_10', 'revenge_JPR7OWTO4Y_1', 'revenge_JPR7OWTO4Y_2', 'revenge_JPR7OWTO4Y_3', 'revenge_JPR7OWTO4Y_4', 'revenge_JPR7OWTO4Y_5', 'enduro_45APTZRP7R_1', 'enduro_45APTZRP7R_2', 'enduro_45APTZRP7R_3', 'seaquest_5E9XSWHWEA_1', 'seaquest_5E9XSWHWEA_2', 'seaquest_5E9XSWHWEA_3', 'seaquest_5E9XSWHWEA_4'])
4433


In [ ]:
# what percentage of frames contain audio utterances

In [ ]:
# what is the breakdown of yes/no utterances per demo

In [1]:
# what is the corresponding reward/return around yes utterances and no utterances
# test speech to text confidence